In [1]:
##Libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
##BBC Chart
bbc = requests.get('https://www.bbc.co.uk/programmes/articles/3tqPdBWF9yMbTrfjWvfKV8t/radio-1-playlist')
soup = BeautifulSoup(bbc.text, 'html.parser')

chart = soup.select('.text--prose')

aList_soup = chart[1].find_all('p', string=True)
bList_soup = chart[2].find_all('p', string=True)
cList_soup = chart[3].find_all('p', string=True)

def cleanData(ls, name):
    df = []
    rank = 1
    name = name
    for i in ls:
        trim = i.get_text()
        if trim[0] == '↑':
            trim = trim[2:]
        tup = (trim.split('-')[0].lower(), trim.split('-')[1].lower(), rank, name)
        df.append(tup)
        rank += 1
    return df

aList = cleanData(aList_soup, 'A-List')
bList = cleanData(bList_soup, 'B-List')
cList = cleanData(cList_soup, 'C-List')

bbc_chart = aList + bList + cList

df1 = pd.DataFrame(bbc_chart, columns=['Artist', 'Song', 'Rank', 'Chart-Name'])


In [3]:
# Official Charts Data
offChart = requests.get('https://www.officialcharts.com/charts/')

chartSoup = BeautifulSoup(offChart.text, 'html.parser')

offChartText = chartSoup.select('.chart-positions')
headings =  []
for i in offChartText[0].find_all('th'):
    headings.append(i.get_text())
    
rows = offChartText[0].find_all('tr')

rowsArr = []
for i in rows:
    cleanRow = i.get_text().replace('\n',' ').replace('\r',' ').replace('   ', ',').replace('  ', ',')
    rowsArr.append(cleanRow)

rowsArr = [i for i in rowsArr if i != ' Pos LW Title, Artist PeakPos WoC,ChartFacts ']
rowsArr = [i for i in rowsArr if i != ',Buy Listen,']
rowsArr = [i for i in rowsArr if i != ',amazon itunes, ']
rowsArr = [i for i in rowsArr if i != ',,']
rowsArr = [i for i in rowsArr if i != ',deezer spotify, ']
rowsArr = [i for i in rowsArr if i != ',spotify deezer, ']
rowsArr = [i for i in rowsArr if i != ',Buy,']
rowsArr = [i for i in rowsArr if i != ',amazon, ']
rowsArr = [i for i in rowsArr if i != ',itunes, ']
rowsArr = [i for i in rowsArr if i != ',spotify, ']
rowsArr = [i for i in rowsArr if i != ',deezer, ']

# Prepare data for DF
officialData = []

for i in rowsArr:
    row = i.split(',')
    cleanRow = list(filter(None, row))
    if len(cleanRow) > 4:
        tup = (cleanRow[3].lower(), cleanRow[2].lower(), cleanRow[0], 'Official Charts')
        officialData.append(tup)

# Create DF
df2 = pd.DataFrame(officialData, columns=['Artist', 'Song', 'Rank', 'Chart-Name'])



In [4]:
# Big top 40


big40Chart = requests.get('https://www.bigtop40.com/')
big40ChartSoup = BeautifulSoup(big40Chart.text, 'html.parser')
big40ChartText = big40ChartSoup.select('.chart-module__charts')

rows = big40ChartText[0].find_all('li')
big40ChartData = []
for i in rows:
    cleanRow = i.get_text().replace('\n','').replace('\t','').replace('                                                            ', ',').replace('                                                    ', ',').split(',')
    big40ChartData.append((cleanRow[3].lower(), cleanRow[1].lower(), cleanRow[0], 'Big top 40'))
                     
            
# Create DF
df3 = pd.DataFrame(big40ChartData, columns=['Artist', 'Song', 'Rank', 'Chart-Name'])

In [5]:
allData = pd.concat([df1, df2, df3], ignore_index=True)
# allData.to_csv('./out.csv')

chars_count = allData['Chart-Name'].value_counts()
allData.reset_index()

points = []
for index, row in allData.iterrows():
    if row['Chart-Name'] == 'Official Charts':
            points.append(int(chars_count['Official Charts']) + 1 - int(row['Rank']))
    elif row['Chart-Name'] == 'A-List':
            points.append(101 + int(chars_count['A-List']) - (int(chars_count['A-List']) + int(row['Rank'])))
    elif row['Chart-Name'] == 'B-List':
            points.append(101 + int(chars_count['B-List']) - (int(chars_count['B-List']) + int(row['Rank'])))
    elif row['Chart-Name'] == 'C-List':
            points.append(101 + int(chars_count['C-List']) - (int(chars_count['C-List']) + int(row['Rank'])))
    elif row['Chart-Name'] == 'Big top 40':
            points.append(101 + int(chars_count['Big top 40']) - (int(chars_count['Big top 40']) +int(row['Rank'])))
            

allData['points'] = points


In [6]:
unique_songs = allData['Song'].value_counts()

total_points = []
artist_list = []
songs_list = []

for index, value in unique_songs.items():
    tot = allData.loc[allData['Song'] == index, 'points'].sum()
    total_points.append(tot)
    artist = allData.iloc[allData[allData['Song'] == index].first_valid_index()]['Artist']
    artist_list.append(artist)
    songs_list.append(index)

lst = zip(artist_list, songs_list, total_points)

final_chart = pd.DataFrame(lst, columns=['Artist', 'Song', 'Combined Points'])
final_chart.sort_values(by=['Combined Points'])
final_chart_sorted = final_chart.sort_values(by=['Combined Points'], ascending=False)


In [7]:
from datetime import date

today = date.today()
file_name = str(today) + '_Chart_Combined.xlsx'

print(file_name)


2023-04-28_Chart_Combined.xlsx


In [8]:
final_chart_sorted.reset_index().to_excel(file_name) 